# 🤖🎥 FacedIt  
Welcome to the **FacedIt Face Swapper Notebook**!  
This notebook uses face detection and image processing to **swap faces in a video**.  

> 📽️ Input: Any video with a visible face  
> 🔄 Output: A new video with a different face  
---


## 📦 Step 1: Install & Import Libraries  
> ⚙️ Setting up the environment...


In [1]:
!pip install insightface
!pip install onnxruntime-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/439.5 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 27.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 106.7 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp311-cp311-linux_x86_64.whl size=1057646 sha256=8db517f53e305a819cc900bea5ceec6c8035716d9ba2b6b52952f3672bc893b7
  Stored in directory: /root/.cache/pip/wheels/27/d8/22/f52d858d16cd06e7b2e6aad34a1777dcfaf000be833bbf8146
Successfully built insightface
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.2/283.2 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.4 MB/s eta 0:00:00


## 🧠 Step 2: Load Face & Video  
> 🎭 Let's load the face to swap and the video in which we will do the magic!

In [2]:
import cv2
import os
import glob
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import insightface
import matplotlib.image as image
import matplotlib.pyplot as plt
import numpy as np
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image
from PIL import Image, ImageTk

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.7'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


> Mount Drive and give model path (download the inswapper_128.onnx model)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Initialising and Setting up model
app = FaceAnalysis(name='buffalo_l', providers=['CUDAExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640,640))
swapper = insightface.model_zoo.get_model('/content/drive/MyDrive/face-swap/inswapper_128.onnx')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:03<00:00, 80409.92KB/s]


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with o

> Create temp folders and define paths


In [4]:
!rm -rf '/content/video_frames'
!rm -rf '/content/processed_images'
!mkdir '/content/video_frames'
!mkdir '/content/processed_images'
!rm -rf '/content/output_video.mp4'

# Source face path
source_face_path="/content/source_face.jpg"
# Path to the input video file
video_path = "/content/input_video.mp4"
# Output directory for the frames
video_frames_path = "/content/video_frames"
# Path to processed frames
processed_frames_path = "/content/processed_images"

## 🧪 Step 3: Face Detection  

> Convert input video to frames

In [5]:
def convert_video_to_frames(video_path, output_dir):
    video = cv2.VideoCapture(video_path)
    frame_count = 0

    while True:
        ret, frame = video.read()
        if not ret:
            break

        output_file = f"{output_dir}/frame_{frame_count:05d}.jpg"
        cv2.imwrite(output_file, frame)

        frame_count += 1

    video.release()

convert_video_to_frames(video_path, video_frames_path)

> Extract facial data of all target frames

In [6]:
target_facial_data = []

def get_target_facial_data(input_dir):
  image_files = [file for file in os.listdir(input_dir) if file.endswith(".jpg")]
  image_files.sort()

  for filename in image_files:
        input_path = os.path.join(input_dir, filename)
        image = cv2.imread(input_path)
        curr_frame_faces = app.get(image)
        target_facial_data.append(curr_frame_faces)

get_target_facial_data(video_frames_path);
total_target_frames = len(target_facial_data)

> Load source face

In [7]:
face1 = None
source_face = cv2.imread(source_face_path)
get_source_face = app.get(source_face)

if(get_source_face == None or len( get_source_face )==0):
  print('No face found in source image')
else:
  face1 = get_source_face[0];

## 🎬 Step 4: Face Swap in Action  
> 🔁 Swapping faces frame by frame...


In [8]:
def swap_face(image, curr_frame_facial_data):
  face2 = None
  get_target_face = curr_frame_facial_data
  swapped_image = image.copy()

  if(get_target_face == None or len( get_target_face )==0):
    print('No target face found in current image')
    return swapped_image
  else:
    face2 = get_target_face[0]

  swapped_image = swapper.get(swapped_image,face2,face1,paste_back=True)
  return swapped_image

> Loop over all frames and call swap face

In [9]:
def process_images(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    image_files = [file for file in os.listdir(input_dir) if file.endswith(".jpg")]
    image_files.sort()
    index = 0

    # Process each image
    for filename in image_files:
        input_path = os.path.join(input_dir, filename)
        image = cv2.imread(input_path)

        # Call swap_face function
        processed_image = swap_face(image, target_facial_data[index])

        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, processed_image)
        print(f"Processed image {index} saved: {output_path}")
        index += 1

input_directory = video_frames_path
output_directory = processed_frames_path

process_images(input_directory, output_directory)

Processed image 0 saved: /content/processed_images/frame_00000.jpg
Processed image 1 saved: /content/processed_images/frame_00001.jpg
Processed image 2 saved: /content/processed_images/frame_00002.jpg
Processed image 3 saved: /content/processed_images/frame_00003.jpg
Processed image 4 saved: /content/processed_images/frame_00004.jpg
Processed image 5 saved: /content/processed_images/frame_00005.jpg
Processed image 6 saved: /content/processed_images/frame_00006.jpg
Processed image 7 saved: /content/processed_images/frame_00007.jpg
Processed image 8 saved: /content/processed_images/frame_00008.jpg
Processed image 9 saved: /content/processed_images/frame_00009.jpg
Processed image 10 saved: /content/processed_images/frame_00010.jpg
Processed image 11 saved: /content/processed_images/frame_00011.jpg
Processed image 12 saved: /content/processed_images/frame_00012.jpg
Processed image 13 saved: /content/processed_images/frame_00013.jpg
Processed image 14 saved: /content/processed_images/frame_

> Generate output video from processed frames

In [10]:
def create_video_from_images(input_dir, output_path, fps):
    image_files = [file for file in os.listdir(input_dir) if file.endswith(".jpg")]
    image_files.sort()

    first_image = cv2.imread(os.path.join(input_dir, image_files[0]))
    height, width, _ = first_image.shape

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Use appropriate codec
    video_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Write each image to the video
    for filename in image_files:
        image_path = os.path.join(input_dir, filename)
        image = cv2.imread(image_path)
        video_writer.write(image)

    video_writer.release()

    print(f"Video saved: {output_path}")

# Specify the input directory containing the processed images
input_directory = processed_frames_path
output_video = "/content/output_video.mp4"

# Specify the frames per second (fps) for the output video
fps = 25
create_video_from_images(input_directory, output_video, fps)

Video saved: /content/output_video.mp4


> Download the output video

In [11]:
from google.colab import files
files.download('/content/output_video.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

> (Additional) Add audio to the output video

In [14]:
!pip install moviepy
from moviepy.editor import VideoFileClip, AudioFileClip, CompositeAudioClip

# Extract audio from input video and append to output video
def extract_audio(video_path, output_audio_path):
    video_clip = VideoFileClip(video_path)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(output_audio_path)
    audio_clip.close()
    video_clip.close()

def add_audio_to_video(video_path, audio_path, output_path):
    video_clip = VideoFileClip(video_path)
    audio_clip = AudioFileClip(audio_path)

    if audio_clip.duration < video_clip.duration:
        audio_clip = audio_clip.subclip(0, video_clip.duration)
    elif audio_clip.duration > video_clip.duration:
        video_duration = video_clip.duration
        audio_duration = audio_clip.duration
        audio_clip = audio_clip.subclip(0, video_duration)

    video_with_audio = video_clip.set_audio(audio_clip)
    video_with_audio.write_videofile(output_path, codec="libx264")
    audio_clip.close()
    video_clip.close()

video_path = video_path
audio_output_path = "/content/audio.wav"
output_video_path = "/content/output_video_with_audio.mp4"

# Extract audio from video
extract_audio(video_path, audio_output_path)

# Add audio to final video
add_audio_to_video('/content/output_video.mp4', audio_output_path, output_video_path)

# Download video with audio
from google.colab import files
files.download('/content/output_video_with_audio.mp4')

MoviePy - Writing audio in /content/audio.wav


MoviePy - Done.


Moviepy - Building video /content/output_video_with_audio.mp4.
MoviePy - Writing audio in output_video_with_audioTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/output_video_with_audio.mp4



Moviepy - Done !
Moviepy - video ready /content/output_video_with_audio.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---  
## ✅ All Done!  
> 🥳 Your video has been processed with the swapped face!  
> ⏬ Check the output and throw a star if you like it!  
